<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/smr_sgdc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv("https://github.com/WilliamPoe/CSCI-290/raw/refs/heads/main/Data/penguins.csv")

In [ ]:
df['species'] = df.species.replace({'Adelie':0, 'Chinstrap':1, 'Gentoo':2})
df['island'] = df.island.replace({'Biscoe':0, 'Dream':1, 'Torgersen':2})
df['sex'] = df.sex.replace({'male':0, 'female':1, 'NA':2})

<ipython-input-28-3c10aee48c28>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['species'] = df.species.replace({'Adelie':0, 'Chinstrap':1, 'Gentoo':2})
<ipython-input-28-3c10aee48c28>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['island'] = df.island.replace({'Biscoe':0, 'Dream':1, 'Torgersen':2})
<ipython-input-28-3c10aee48c28>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt

In [ ]:
train_set, test_set= train_test_split(df, test_size=0.2)

In [ ]:
train_set = train_set[['species', 'island', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex', 'year']]
test_set = test_set[['species', 'island', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex', 'year']]

In [ ]:
X_train = train_set[['species', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex', 'year']]
y_train = train_set['island']
X_test = test_set[['species', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex', 'year']]
y_test = test_set['island']

In [ ]:
num_attributes = X_train.select_dtypes( include = ['float64','int64']).columns
## This might be empty!? ##
cat_attributes = X_train.select_dtypes( include = ['object']).columns

In [ ]:
trf = [ ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
         ('Scaler', StandardScaler())]),
         num_attributes), ('cat', OneHotEncoder( handle_unknown='ignore' ), cat_attributes) ]
col_transform = ColumnTransformer( transformers = trf )

# Softmax Regression

# Stochastic Gradient Descent Classifier